# Test the `map()` Function to Format Datasets

## Install and Import Dependencies

In [ ]:
!pip install -qU datasets pprint

In [ ]:
from datasets import load_dataset
import pprint
import torch

## Load Dataset

In [ ]:
dataset = load_dataset("claudios/code_search_net", "go", split="train[:10]")

In [ ]:
print(dataset)

## Define Prompt Format Function for a Single Sample

In [ ]:
# # EOS Token is required to stop open-ended generation and eventual hallucination
# EOS_TOKEN = tokenizer.eos_token 

In [ ]:
def formatFunctionSample(sample):
    language = sample['language']
    instruction = f"What does this {language} function do?"
    inputText = sample['func_code_string']
    outputText = sample['func_documentation_string']

    return {
        "instruction": instruction,
        "input": inputText,
        "output": outputText
    }

## Use the `map()` Method to Format the Dataset

In [ ]:
formattedDataset = dataset.map(formatFunction, remove_columns=dataset.column_names)

## Sample Formatted Dataset

In [ ]:
len(formattedDataset)

In [ ]:
pprint.pp(formattedDataset[1])